In [51]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from keras import metrics
import random
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import cv2
import PIL
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tqdm import tqdm

In [65]:
# load crop data pairs

# Paths to your data.
pos_path = './archive/cropped/CFD_background_region/'
neg_path = './archive/cropped/CFD_background_region/'
img_size = (112, 112)

# Load and preprocess data
def load_data_pairs(pos_path, neg_path):
    pos_images= []  # List to store the images
    neg_images = []   # List to store the masks
    pairList = []
    files = []
    count = 0
    for filename in os.listdir(pos_path):
        if count > 96:
            break
        pos_img_path = pos_path + filename
        neg_img_path = neg_path + filename  # Assuming mask has same filename
        # print(img_path)
        # Load and preprocess the image
        pos_img = cv2.imread(pos_img_path)
        neg_img = cv2.imread(neg_img_path)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        pos_img, neg_img = pos_img/255.0,  neg_img/255.0
        # img = cv2.resize(img, img_size) / 255.0
        # # Load and preprocess the mask using OpenCV
        # mask = cv2.resize(mask, img_size) / 255.0
 
        # for i in range(4):
        #     for j in range(4):
        #         img_part = pos_img[i * 112: (i + 1) * 112,
        #                     j * 112: (j + 1) * 112]
        #         mask_part = neg_img[i * 112: (i + 1) * 112,
        #                     j * 112: (j + 1) * 112]
        pos_images.append(pos_img)
        neg_images.append(neg_img)
        files.append(filename)
         
        count += 1
 
    return np.array(pos_images), np.array(neg_images)
 

pos_img, neg_img = load_data_pairs(pos_path, neg_path)
pos_ds = tf.data.Dataset.from_tensor_slices(pos_img)
neg_ds = tf.data.Dataset.from_tensor_slices(neg_img)

# Zip the positive and negative datasets together to create pairs
train_dataset = tf.data.Dataset.zip((pos_ds, neg_ds))

# Shuffle and batch the dataset as needed
buffer_size = len(pos_img)  # This should be the number of images, can be replaced with a suitable buffer size
batch_size = 2  # Define your batch size as needed
train_dataset = train_dataset.shuffle(buffer_size=buffer_size).batch(batch_size)

# Prefetch data for performance optimization
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
 

improved unet

In [29]:
# Residual block
# def residual_block(input_tensor, num_filters):
#     x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)

#     x = layers.Conv2D(num_filters, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)

#     # Add the input_tensor (residual connection) to the output of the convolution block
#     x = layers.add([x, input_tensor])
#     x = layers.ReLU()(x)

#     return x
def residual_block(input_tensor, num_filters):
    # Check if a projection shortcut is needed
    input_shape = input_tensor.shape[-1]
    if input_shape != num_filters:
        shortcut = layers.Conv2D(num_filters, (1, 1), strides=(1, 1), padding='same')(input_tensor)
    else:
        shortcut = input_tensor

    x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Add the shortcut (which has been adjusted if necessary) to the output of the convolution block
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)

    return x
# Attention gate
def attention_gate(input_tensor, gate_tensor, num_filters):
    # Resize gate tensor to match the input_tensor shape using a 1x1 convolution
    gate_resized = layers.Conv2D(num_filters, 1, padding='same')(gate_tensor)
    gate_resized = layers.BatchNormalization()(gate_resized)
    gate_resized = layers.ReLU()(gate_resized)

    # Add the gate to the input_tensor
    x = layers.add([input_tensor, gate_resized])
    attention = layers.Conv2D(num_filters, 1, padding='same', activation='sigmoid')(x)
    x = layers.multiply([input_tensor, attention])

    return x

def conv_block(input_tensor, num_filters):
    x = residual_block(input_tensor, num_filters)
    return x

def encoder_block(input_tensor, num_filters, block_name):
    x = layers.Conv2D(num_filters, (3, 3), activation='relu', padding='same', name=f"{block_name}_conv")(input_tensor)
    x = layers.BatchNormalization(name=f"{block_name}_bn")(x)
    p = layers.MaxPooling2D((2, 2), name=f"{block_name}_pool")(x)
    return x, p
# def encoder_block(input_tensor, num_filters):
#     x = conv_block(input_tensor, num_filters)
#     p = layers.MaxPooling2D((2, 2))(x)
#     return x, p

# Modify the decoder_block to include an attention gate
def decoder_block(input_tensor, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_tensor)

    # Apply attention gate to the skip features before concatenating
    attention_skipped = attention_gate(skip_features, x, num_filters)

    # Concatenate upsampled input with the attention modified skip features
    x = layers.concatenate([x, attention_skipped])
    x = conv_block(x, num_filters)
    return x

# def build_unet(input_shape):
#     inputs = layers.Input(input_shape)

#     # Encoder
#     s1, p1 = encoder_block(inputs, 64)
#     s2, p2 = encoder_block(p1, 128)
#     s3, p3 = encoder_block(p2, 256)
#     s4, p4 = encoder_block(p3, 512)

#     # Bridge
#     b1 = conv_block(p4, 1024)

#     # Decoder
#     d1 = decoder_block(b1, s4, 512)
#     d2 = decoder_block(d1, s3, 256)
#     d3 = decoder_block(d2, s2, 128)
#     d4 = decoder_block(d3, s1, 64)

#     # Output
#     outputs = layers.Conv2D(1, (1,1), padding="same", activation="sigmoid")(d4)

#     model = models.Model(inputs, outputs)
#     return model

# input_shape = (112, 112, 3)
# model = build_unet(input_shape)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


encoder

In [30]:
# def build_encoder(input_shape):
#     inputs = layers.Input(shape=input_shape)
#     x, _ = encoder_block(inputs, 64)
#     x, _ = encoder_block(x, 128)
#     x, _ = encoder_block(x, 256)
#     x, _ = encoder_block(x, 512)
#     x = conv_block(x, 1024)
#     x = layers.GlobalAveragePooling2D()(x)
#     return keras.Model(inputs, x, name="encoder")
def build_encoder(input_shape):
    inputs = layers.Input(shape=input_shape)
    s1, p1 = encoder_block(inputs, 64, "pre1")
    s2, p2 = encoder_block(p1, 128, "pre2")
    s3, p3 = encoder_block(p2, 256, "pre3")
    s4, p4 = encoder_block(p3, 512, "pre4")
 
    x = layers.GlobalAveragePooling2D()(p4)
    return keras.Model(inputs, x, name="encoder")

# Define the projection head using the functional API
def build_projection_head(encoder):
    inputs = layers.Input(shape=(512,))
    x = layers.Dense(512, activation="relu")(inputs)
    outputs = layers.Dense(256)(x)
    return keras.Model(inputs, outputs, name="projection_head")

# Get the models
input_shape = (112, 112, 3)
encoder_model = build_encoder(input_shape)
projection_head_model = build_projection_head(encoder_model)

contrastive_loss

In [31]:
import tensorflow as tf

# Cosine similarity function
def cosine_similarity(z1, z2):
    z1 = tf.math.l2_normalize(z1, axis=1)
    z2 = tf.math.l2_normalize(z2, axis=1)
    return tf.matmul(z1, z2, transpose_b=True)

def contrastive_loss(z_crack, z_bg, other_proj, temperature=0.5):
    # Normalize the input projections
    z_crack_norm = tf.math.l2_normalize(z_crack, axis=1)
    z_bg_norm = tf.math.l2_normalize(z_bg, axis=1)
    other_proj_norm = tf.math.l2_normalize(other_proj, axis=1)
 
    # Compute the positive similarity scores
    pos_sim = tf.reduce_sum(z_crack_norm * z_bg_norm, axis=1, keepdims=True)
    exp_pos_sim = tf.exp(pos_sim / temperature)

    # Compute the negative similarity scores
    neg_sim_z_crack = tf.matmul(z_crack_norm, other_proj_norm, transpose_b=True)
    neg_sim_z_bg = tf.matmul(z_bg_norm, other_proj_norm, transpose_b=True)

    # Sum the exponentials of the negative similarities
    sum_exp_neg_sim_z_crack = tf.reduce_sum(tf.exp(neg_sim_z_crack / temperature), axis=1, keepdims=True)
    sum_exp_neg_sim_z_bg = tf.reduce_sum(tf.exp(neg_sim_z_bg / temperature), axis=1, keepdims=True)

    # Calculate the loss for the positive pair with respect to all other projections
    loss_z_crack = -tf.math.log(exp_pos_sim / (sum_exp_neg_sim_z_crack + exp_pos_sim + 1e-9)) # Add epsilon for numerical stability
    loss_z_bg = -tf.math.log(exp_pos_sim / (sum_exp_neg_sim_z_bg + exp_pos_sim + 1e-9))

    # Combine the losses for both members of the positive pair
    loss_per_pair = loss_z_crack + loss_z_bg

    # Average the loss across all positive pairs
    loss = tf.reduce_mean(loss_per_pair)

    return loss


In [67]:

inputs = tf.keras.Input(shape=(112, 112, 3))

# Forward pass through the encoder and projection head
encoder = build_encoder(input_shape=(112, 112, 3))
encoder_output_shape = encoder.output_shape[1]
projection_head = build_projection_head(encoder_output_shape)

encoded = encoder(inputs)
projected = projection_head(encoded)

# Define the Keras model
model = tf.keras.Model(inputs=inputs, outputs=projected)

# Compile the model with a dummy optimizer and loss since we are using a custom training loop
model.compile(optimizer='adam', loss=None)
num_epochs = 3
optimizer = tf.keras.optimizers.Adam()

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    pbar = tqdm(total=len(train_dataset), desc='Training', leave=True)
    
    for step, (images_crack, images_bg) in enumerate(train_dataset):
        # Open the GradientTape context to capture gradients
        with tf.GradientTape() as tape: 
            # Compute all projections first
            proj_crack = model(images_crack, training=True)  # Projections of crack images
            proj_bg = model(images_bg, training=True)  # Projections of background images
            all_projections = tf.concat([proj_crack, proj_bg], axis=0)

            # Calculate the loss for each image
            loss_values = []
            for i in range(tf.shape(proj_crack)[0]):
                # Positive pair for the current sample
                pos_pair = tf.concat([proj_crack[i:i+1], proj_bg[i:i+1]], axis=0)
                other_bg = tf.concat([proj_bg[0:i], proj_bg[i+1:]], axis=0)
                other_crack = tf.concat([proj_crack[0:i], proj_crack[i+1:]], axis=0)
                other_pj = tf.concat([other_bg[:], other_crack[:]], axis = 0)
                # Compute the loss for the current pair against all other projections
                loss_value = contrastive_loss(pos_pair[0:1], pos_pair[1:2], other_pj, temperature=0.5)
                loss_values.append(loss_value)

            # Calculate the mean of the loss values for gradient update
            batch_loss = tf.reduce_mean(tf.stack(loss_values))

        # Calculate gradients and update model weights
        gradients = tape.gradient(batch_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        pbar.update(1)
        pbar.set_postfix(loss=f"{batch_loss.numpy():.4f}")

    pbar.close()


Epoch 1/3


Training: 100%|██████████| 49/49 [00:42<00:00,  1.14it/s, loss=-0.0000]


Epoch 2/3


Training: 100%|██████████| 49/49 [00:41<00:00,  1.19it/s, loss=-0.0000]


Epoch 3/3


Training: 100%|██████████| 49/49 [00:42<00:00,  1.16it/s, loss=-0.0000]


In [33]:
# encoder_layers = [layer for layer in encoder_model.layers if 'conv' in layer.name or 'pooling' in layer.name]
# model_encoder_layers = [layer for layer in model.layers if 'conv' in layer.name or 'pooling' in layer.name][:len(encoder_layers)]
# for unet_layer, encoder_layer in zip(model_encoder_layers, encoder_layers):
#     unet_layer.set_weights(encoder_layer.get_weights())

In [72]:
def decoder_block(input_tensor, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_tensor)
    attention_skipped = attention_gate(skip_features, x, num_filters)  # Apply attention gate
    x = layers.concatenate([x, attention_skipped])
    x = conv_block(x, num_filters)
    return x

# Function to build the entire U-Net model
def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64, "encoder1")
    s2, p2 = encoder_block(p1, 128, "encoder2")
    s3, p3 = encoder_block(p2, 256, "encoder3")
    s4, p4 = encoder_block(p3, 512, "encoder4")


    # Bridge
    b1 = conv_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = layers.Conv2D(1, (1,1), padding="same", activation="sigmoid")(d4)

    model = models.Model(inputs, outputs)
    return model

 

# Build the U-Net model
input_shape = (112, 112, 3)
unet_model = build_unet(input_shape)
for layer in unet_model.layers:
    if 'enoder' in layer.name:  # Only load weights for the encoder layers
        layer_name = "pre" + layer.name[-1]  # Assumes layer names are like "block1_conv"
        layer_weights = encoder.get_layer(layer_name).get_weights()
        layer.set_weights(layer_weights)

# Compile the U-Net model
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - (numerator + 1) / (denominator + 1)

unet_model.compile(optimizer='adam',
                   loss=dice_loss,
                   metrics=[metrics.Precision(),
                            metrics.Recall(),
                            metrics.FalsePositives(),
                            metrics.FalseNegatives(),
                            metrics.BinaryIoU()])

# 

In [53]:

# Paths to your data.
images_path = './archive/crack_segmentation_dataset/train/images/'
masks_path = './archive/crack_segmentation_dataset/train/masks/'
img_size = (112, 112)

# Load and preprocess data
def load_data(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    files = []
    count = 0
    for filename in os.listdir(images_path):
        if count > 500:
            break
        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename
        # print(img_path)
        # Load and preprocess the image
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        img, mask = img/255.0, mask/255.0
        # img = cv2.resize(img, img_size) / 255.0
        # # Load and preprocess the mask using OpenCV
        # mask = cv2.resize(mask, img_size) / 255.0
 
        for i in range(4):
            for j in range(4):
                img_part = img[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                mask_part = mask[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                images.append(img_part)
                masks.append(mask_part)
        files.append(filename)
         
        count += 1
    print(files)
    return np.array(images), np.array(masks)


In [74]:
def load_test(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    count = 0
    print()
    imgList = os.listdir(images_path)
    files = []
    for i in range(len(imgList) - 1, -1, -1):
        filename = imgList[i]
    # for filename in os.listdir(images_path):
        if count >= 100:
            break

        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        img, mask = img/255.0, mask/255.0

        i, j = random.randint(0, 3), random.randint(0, 3)
        img_part = img[i * 112: (i + 1) * 112,
                    j * 112: (j + 1) * 112]
        mask_part = mask[i * 112: (i + 1) * 112,
                    j * 112: (j + 1) * 112]
        images.append(img_part)
        masks.append(mask_part)

        files.append(filename)
        count += 1
    print(files)
    return np.array(images), np.array(masks)

In [76]:
# Load your data
images, masks = load_data(images_path, masks_path, (112, 112))


['CFD_002.jpg', 'CFD_003.jpg', 'CFD_004.jpg', 'CFD_005.jpg', 'CFD_006.jpg', 'CFD_008.jpg', 'CFD_009.jpg', 'CFD_010.jpg', 'CFD_012.jpg', 'CFD_015.jpg', 'CFD_016.jpg', 'CFD_017.jpg', 'CFD_018.jpg', 'CFD_020.jpg', 'CFD_021.jpg', 'CFD_022.jpg', 'CFD_023.jpg', 'CFD_024.jpg', 'CFD_025.jpg', 'CFD_026.jpg', 'CFD_027.jpg', 'CFD_028.jpg', 'CFD_029.jpg', 'CFD_030.jpg', 'CFD_031.jpg', 'CFD_032.jpg', 'CFD_033.jpg', 'CFD_034.jpg', 'CFD_035.jpg', 'CFD_036.jpg', 'CFD_038.jpg', 'CFD_039.jpg', 'CFD_041.jpg', 'CFD_042.jpg', 'CFD_043.jpg', 'CFD_044.jpg', 'CFD_045.jpg', 'CFD_046.jpg', 'CFD_048.jpg', 'CFD_049.jpg', 'CFD_050.jpg', 'CFD_051.jpg', 'CFD_052.jpg', 'CFD_053.jpg', 'CFD_054.jpg', 'CFD_055.jpg', 'CFD_056.jpg', 'CFD_057.jpg', 'CFD_058.jpg', 'CFD_059.jpg', 'CFD_060.jpg', 'CFD_061.jpg', 'CFD_062.jpg', 'CFD_063.jpg', 'CFD_064.jpg', 'CFD_065.jpg', 'CFD_066.jpg', 'CFD_067.jpg', 'CFD_068.jpg', 'CFD_069.jpg', 'CFD_071.jpg', 'CFD_072.jpg', 'CFD_073.jpg', 'CFD_074.jpg', 'CFD_075.jpg', 'CFD_076.jpg', 'CFD_077.

MemoryError: Unable to allocate 2.25 GiB for an array with shape (8016, 112, 112, 3) and data type float64

In [ ]:
images.shape

(8016, 112, 112, 3)

In [ ]:
images1, masks1 = images[:500], masks[:500]
masks1 = np.expand_dims(masks1, axis=-1)

In [58]:

# Load your data
imgval, maskval = load_test(images_path, masks_path, (112, 112))

maskval = np.expand_dims(maskval, axis=-1)


['Volker_DSC01710_725_197_1392_1586.jpg', 'Volker_DSC01710_608_6_1524_1817.jpg', 'Volker_DSC01710_521_437_1123_892.jpg', 'Volker_DSC01710_449_161_1824_1502.jpg', 'Volker_DSC01710_37_518_1605_1304.jpg', 'Volker_DSC01710_340_159_2049_1635.jpg', 'Volker_DSC01710_15_28_1388_1132.jpg', 'Volker_DSC01710_112_246_1993_1548.jpg', 'Volker_DSC01710_1087_22_1502_1742.jpg', 'Volker_DSC01710_1025_76_1293_1327.jpg', 'Volker_DSC01710_0_0_2736_1824.jpg', 'Volker_DSC01709_599_1337_1214_1373.jpg', 'Volker_DSC01709_525_1036_1256_1448.jpg', 'Volker_DSC01709_43_382_1337_1425.jpg', 'Volker_DSC01709_1_428_1791_1806.jpg', 'Volker_DSC01709_191_287_1561_2056.jpg', 'Volker_DSC01709_176_298_1435_1635.jpg', 'Volker_DSC01709_14_103_1566_1955.jpg', 'Volker_DSC01709_109_399_1680_2130.jpg', 'Volker_DSC01709_0_0_1824_2736.jpg', 'Volker_DSC01708_820_247_911_1098.jpg', 'Volker_DSC01708_7_339_1805_2017.jpg', 'Volker_DSC01708_6_308_1788_2155.jpg', 'Volker_DSC01708_3_1033_1692_1349.jpg', 'Volker_DSC01708_26_342_1730_1845.jp

In [59]:
masks1.shape

(500, 112, 112, 1)

In [60]:

X_train, y_train, X_val, y_val = images1, masks1,imgval, maskval 
# y_train.shape
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (500, 112, 112, 3)
y_train shape: (500, 112, 112, 1)
X_val shape: (101, 112, 112, 3)
y_val shape: (101, 112, 112, 1)


In [61]:
images1[:200,:,:,:].shape
 

(200, 112, 112, 3)

In [73]:


# Create data generators with desired augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Generate batches of augmented data from arrays (X_train and y_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=8)
val_generator = val_datagen.flow(X_val, y_val, batch_size=8)

# unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model using the fit_generator function
history = unet_model.fit(
    train_generator, 
    steps_per_epoch=len(X_train) // 8,  # Number of batches per epoch
    epochs=10, 
    validation_data=val_generator,
    validation_steps=len(X_val) // 8  # Number of batches for validation
)


Epoch 1/10


c:\Users\link1\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - binary_io_u_3: 0.4570 - false_negatives_3: 70948.0312 - false_positives_3: 271644.4688 - loss: 0.8851 - precision_3: 0.1048 - recall_3: 0.3594 - val_binary_io_u_3: 0.4727 - val_false_negatives_3: 52705.0000 - val_false_positives_3: 51801.0000 - val_loss: 0.9539 - val_precision_3: 0.0460 - val_recall_3: 0.0452
Epoch 2/10
 1/62 ━━━━━━━━━━━━━━━━━━━━ 1:06 1s/step - binary_io_u_3: 0.4825 - false_negatives_3: 461.0000 - false_positives_3: 5288.0000 - loss: 0.9310 - precision_3: 0.0736 - recall_3: 0.4767

c:\Users\link1\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - binary_io_u_3: 0.4825 - false_negatives_3: 461.0000 - false_positives_3: 5288.0000 - loss: 0.4730 - precision_3: 0.0736 - recall_3: 0.4767 - val_binary_io_u_3: 0.4868 - val_false_negatives_3: 0.0000e+00 - val_false_positives_3: 1660.0000 - val_loss: 0.4998 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - binary_io_u_3: 0.5157 - false_negatives_3: 70921.6797 - false_positives_3: 109170.7656 - loss: 0.8198 - precision_3: 0.2154 - recall_3: 0.2954 - val_binary_io_u_3: 0.3551 - val_false_negatives_3: 34215.0000 - val_false_positives_3: 352629.0000 - val_loss: 0.9240 - val_precision_3: 0.0519 - val_recall_3: 0.3608
Epoch 4/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - binary_io_u_3: 0.5220 - false_negatives_3: 3100.0000 - false_positives_3: 4047.0000 - loss: 0.3783 - precision_3: 0.2822 - recall_3: 0.3392 - val_binary_io_u_3: 0.3172 - val_false_negatives_3: 1016.0000 - val_false_positives_3: 2

In [64]:


# Create data generators with desired augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator()  # No augmentation for validation data

 
for layer in model.layers[:4]:
    layer.trainable = False
    print(layer.name)
    
train_generator = train_datagen.flow(X_train, y_train, batch_size=8)
val_generator = val_datagen.flow(X_val, y_val, batch_size=8)

# unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model using the fit_generator function
history = unet_model.fit(
    train_generator, 
    steps_per_epoch=len(X_train) // 8,  # Number of batches per epoch
    epochs=10, 
    validation_data=val_generator,
    validation_steps=len(X_val) // 8  # Number of batches for validation
)

input_layer_21
encoder
projection_head
Epoch 1/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 62s 1000ms/step - binary_io_u: 0.4935 - false_negatives: 105416.0938 - false_positives: 0.0000e+00 - loss: 0.0821 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_binary_io_u: 0.4865 - val_false_negatives: 55200.0000 - val_false_positives: 0.0000e+00 - val_loss: 0.1916 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - binary_io_u: 0.4902 - false_negatives: 4938.0000 - false_positives: 0.0000e+00 - loss: 0.0671 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_binary_io_u: 1.0000 - val_false_negatives: 0.0000e+00 - val_false_positives: 0.0000e+00 - val_loss: 0.0039 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
62/62 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - binary_io_u: 0.4933 - false_negatives: 105811.5859 - false_positives: 0.8889 - loss: 0.0835 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_binary_io_u: 0.4869 - val_false_negatives: 537

In [ ]:

# Load and preprocess the image
image_path = 'CFD_001.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (112, 112))  # Resize to match your model's input shape
# image = image / 255.0  # Normalize the pixel values
cv2.imwrite( "asg.jpg", image)
# Convert the image to a NumPy array with the expected shape
# input_image = np.expand_dims(image, axis=0)  # Add a batch dimension
prediction = model.predict(input_image)
prediction

NameError: name 'input_image' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

prediction *= 255

prediction_2d = prediction.squeeze()

plt.figure(figsize=(8, 8))
plt.imshow(prediction_2d, cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()

NameError: name 'prediction' is not defined